
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
   
                                          
                                          
                                          
                                          
                                          
#                                                 Regression with Abalone Data Set: Random Forest and XG BOOST Models




#                                                        Gladys Murage

#                              College of Business, Engineering, and  Technology, National University

#                                         DDS8555 v1: PREDICTIVE ANALYSIS(3602869492)

#                                                        Dr MOHAMED NABEEL

#                                                            March 06, 2025


<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>
<BR>



# Model 2 is Random Forest

In [19]:
# Import necessary libraries
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pandas as pd

# Load the datasets
train_data = pd.read_csv("train.csv")
test_data = pd.read_csv("test.csv")
sample_submission = pd.read_csv("sample_submission.csv")

# Identify categorical columns (assuming they are object dtype)
categorical_columns = train_data.select_dtypes(include=['object']).columns

# Convert categorical variables to numerical using one-hot encoding
# Remove the target variable 'Rings' and apply one-hot encoding
X = pd.get_dummies(train_data.drop('Rings', axis=1))  # Keep this line
y = train_data['Rings']

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Initialize and train Random Forest model
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train_scaled, y_train)

# Make predictions on the validation set
y_val_pred_rf = rf_model.predict(X_val_scaled)

# Define RMSLE function
def rmsle(y_true, y_pred):
    import numpy as np
    return np.sqrt(np.mean(np.power(np.log1p(y_pred) - np.log1p(y_true), 2)))

# Evaluate the Random Forest model
rmsle_value_rf = rmsle(y_val, y_val_pred_rf)
print(f"Validation RMSLE (Random Forest): {rmsle_value_rf}")

Validation RMSLE (Random Forest): 0.15567026949474086


In [ ]:
# Create a submission DataFrame
submission = pd.DataFrame({
    "id": test_data["id"],  
    "Rings": y_pred
})

# Save the submission file
submission.to_csv("submission2.csv", index=False)

print("Submission file created successfully!")


# Results with Random Forest RMSLE = 0.1557
This shows some improvement but a better model to evaluate RMSLE is called for. I shall now perform XGBoost.

# XGBoost is a powerful gradient boosting algorithm
XG Boost is an algorithim that often outperforms Random Forest and Linear Regression, especially in structured data problems. Below is the code incorporating XGBoost with hyperparameter tuning.

# Why XGBoost?
✅ Handles missing values automatically
✅ No need for feature scaling
✅ Often outperforms Random Forest in structured data
✅ Supports parallelization and GPU acceleration

# Key Features in This Code
1. Raw Features Used: XGBoost does not require feature scaling.
2. Hyperparameter Tuning: Uses GridSearchCV for optimal parameters.
3. Log-Transformation Option: Helps with target variable skewness.
4. Expected Outcome:

    a) A Likely lower RMSLE compared to Linear Regression and Random Forest.

    b) Faster training with optimized parameters.

In [1]:
!pip install xgboost


Defaulting to user installation because normal site-packages is not writeable
Looking in links: /usr/share/pip-wheels


In [2]:
# Import libraries
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_log_error
from xgboost import XGBRegressor

# Define RMSLE function
def rmsle(y_true, y_pred):
    """
    Calculate Root Mean Squared Logarithmic Error (RMSLE).
    """
    y_pred = np.maximum(y_pred, 0)  # Ensure non-negative predictions
    return np.sqrt(mean_squared_log_error(y_true, y_pred))

# Load the training data
train_data = pd.read_csv("train.csv")

# Handle missing values (better than dropping all rows)
# Fill missing values only for numeric columns
numeric_cols = train_data.select_dtypes(include=['number']).columns
train_data[numeric_cols] = train_data[numeric_cols].fillna(train_data[numeric_cols].median())

# One-hot encode categorical variables
train_data = pd.get_dummies(train_data, columns=["Sex"], drop_first=True)

# Separate features and target
X = train_data.drop("Rings", axis=1)
y = train_data["Rings"]

# Split the data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardization is NOT needed for XGBoost, but we can try both ways
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Train an XGBoost model
xgb_model = XGBRegressor(objective="reg:squarederror", random_state=42, n_estimators=100)
xgb_model.fit(X_train, y_train)  # Using raw data (XGBoost handles scaling internally)

# Make predictions
y_val_pred_xgb = xgb_model.predict(X_val)

# Evaluate XGBoost using RMSLE
rmsle_value_xgb = rmsle(y_val, y_val_pred_xgb)
print(f"Validation RMSLE (XGBoost): {rmsle_value_xgb:.4f}")

# Hyperparameter tuning for XGBoost
param_grid_xgb = {
    'n_estimators': [100, 200, 300],
    'max_depth': [3, 6, 9],
    'learning_rate': [0.01, 0.05, 0.1],
    'subsample': [0.7, 0.8, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Perform Grid Search with cross-validation
grid_search_xgb = GridSearchCV(XGBRegressor(objective="reg:squarederror", random_state=42),
                               param_grid_xgb, scoring="neg_mean_squared_log_error", cv=5, verbose=1)
grid_search_xgb.fit(X_train, y_train)

# Get the best XGBoost model
best_xgb_model = grid_search_xgb.best_estimator_

# Make predictions using the best model
y_val_pred_best_xgb = best_xgb_model.predict(X_val)

# Evaluate the best XGBoost model using RMSLE
rmsle_value_best_xgb = rmsle(y_val, y_val_pred_best_xgb)
print(f"Validation RMSLE (Best XGBoost Model): {rmsle_value_best_xgb:.4f}")

# Log Transformation of the Target (Optional)
y_train_log = np.log1p(y_train)
y_val_log = np.log1p(y_val)

# Train and evaluate the XGBoost model with log-transformed target
best_xgb_model.fit(X_train, y_train_log)
y_val_pred_log_xgb = best_xgb_model.predict(X_val)
y_val_pred_xgb_final = np.expm1(y_val_pred_log_xgb)

# Evaluate transformed model
rmsle_value_log_xgb = rmsle(y_val, y_val_pred_xgb_final)
print(f"Validation RMSLE (Log-Transformed Target, XGBoost): {rmsle_value_log_xgb:.4f}")


Validation RMSLE (XGBoost): 0.1531
Fitting 5 folds for each of 243 candidates, totalling 1215 fits
Validation RMSLE (Best XGBoost Model): 0.1521
Validation RMSLE (Log-Transformed Target, XGBoost): 0.1514


# Prepare the Submission File
Format the predictions to match the sample submission file. Typically, the submission file requires an ID column and the predicted target variable.

In [20]:
## Create a submission DataFrame
submission = pd.DataFrame({
    "id": test_data["id"],  # Assuming the test data has an 'id' column
    "Rings": y_pred
})

# Save the submission file
submission.to_csv("submission3.csv", index=False)

print("Submission 3 file created successfully!")

Submission 3 file created successfully!


# Verify the Submission File
Ensure the submission file matches the format of the sample submission file.

In [26]:
# Load the submission file into a DataFrame
submission3 = pd.read_csv("submission3.csv")

# Print the top 5 records
print(submission3.head())

      id      Rings
0  90615   9.930454
1  90616   9.582545
2  90617  10.162455
3  90618  10.875250
4  90619   7.533986


In [30]:
# Print the last 5 records
print(submission3.tail())

           id      Rings
60406  151021   6.427125
60407  151022   9.437661
60408  151023  12.048299
60409  151024  13.385188
60410  151025   8.682507


In [34]:
# Print the shape of the submission 3 DataFrame
print("Shape of submission file:", submission3.shape)

Shape of submission file: (60411, 2)


# The results obtained show the Validation RMSLE for multiple models and steps in the workflow. 
A detailed interpretation of these results and what they mean for the model's performance:

# 1. Summary of Results
Model / Step	Validation RMSLE

Linear Regression	0.1668

Random Forest	0.1557

XGBoost (Default Hyperparameters)	0.1531

XGBoost (Best Model from Grid Search)	0.1521

XGBoost (Log-Transformed Target)	0.1514

# 2. Interpretation of Results
## Linear Regression (RMSLE: 0.1668)
The Linear Regression model has the highest RMSLE, indicating that it is the least accurate among the models tested.

This is expected because Linear Regression is a simple model and may not capture complex patterns in the data.

## Random Forest (RMSLE: 0.1557)
The Random Forest model performs better than Linear Regression, with a lower RMSLE of 0.1557.

Random Forest is more flexible and can capture non-linear relationships in the data, which explains the improvement.

## XGBoost (Default Hyperparameters, RMSLE: 0.1531)
The XGBoost model with default hyperparameters has a lower RMSLE (0.1531) than Random Forest, making it the best-performing model so far.

XGBoost is a powerful ensemble method that often outperforms Random Forest due to its gradient-boosting framework and regularization techniques.

## XGBoost (Best Model from Grid Search, RMSLE: 0.1521)
After hyperparameter tuning using Grid Search, the best XGBoost model achieves a slightly lower RMSLE of 0.1521.

This improvement shows that hyperparameter tuning can help optimize the model's performance.

## XGBoost (Log-Transformed Target, RMSLE: 0.1514)
Applying a log transformation to the target variable further reduces the RMSLE to 0.1514.

Log transformation helps handle skewness in the target variable, making the model more robust to extreme values.

# 3. Key Insights
## a) Model Complexity Matters:

More complex models (Random Forest, XGBoost) outperform simpler models (Linear Regression) because they can capture non-linear relationships in the data.

## b) Hyperparameter Tuning Helps:

Grid Search improves the XGBoost model's performance by finding the best hyperparameters.

## C) Log Transformation Improves Performance:

Log-transforming the target variable reduces skewness and further improves the model's performance.

## d) Room for Improvement:

While the RMSLE has improved significantly, there is still room for further improvement (e.g., feature engineering, advanced models). 

# 4. Next Steps
## Step 1: Feature Engineering
Create new features (e.g., interaction terms, polynomial features).

Transform features (e.g., log transformation, scaling).

## Step 2: Try Advanced Models
LightGBM: A faster and more efficient gradient-boosting framework.

CatBoost: Handles categorical features natively and is robust to overfitting.

Neural Networks: For highly complex datasets.

## Step 3: Ensemble Methods
Combine multiple models (e.g., XGBoost, Random Forest) to improve performance.

## Step 4: Handle Outliers
Identify and handle outliers in the target variable or features.

## Step 5: Cross-Validation
Use cross-validation to ensure the model's performance is consistent across different subsets of the data.